In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load the CSV file
df = pd.read_csv('validator_data.csv')

# Function to create lag features
def create_lag_features(df, lag=1):
    for i in range(1, lag + 1):
        df[f'lag_{i}'] = df['effectiveness'].shift(i)
    return df

# Create lag features for the past 5 days
df = df.groupby('id').apply(create_lag_features, lag=5)
df = df.dropna().reset_index(drop=True)
# print(df.head())
 
# Define features and target
# features = [col for col in df.columns if 'lag_' in col]
features = ['total_reward']
target = 'effectiveness'

# Split the data into training and testing sets
train_df = df[df['partition_date'] < '2024-05-02']
test_df = df[df['partition_date'] >= '2024-05-02']

X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]
y_test = test_df[target]

# Initialize and train the XGBoost model
model = LinearRegression()
model.fit(X_train, y_train)

/var/folders/pj/vyqltmgj79xghgpb38_96m040000gn/T/ipykernel_39972/115765728.py:17: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('id').apply(create_lag_features, lag=5)


LinearRegression()

In [3]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Define the initial types for the ONNX model
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]

# Convert the scikit-learn model to ONNX
onnx_model = convert_sklearn(model, initial_types=initial_type)

# Save the ONNX model to a file
with open("linear_regression.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [4]:
!giza transpile linear_regression.onnx --output-path verifiable_lr

[giza][2024-06-09 13:25:12.753] No model id provided, checking if model exists ✅
[giza][2024-06-09 13:25:12.755] Model name is: linear_regression
[giza][2024-06-09 13:25:14.302] Model Created with id -> 764! ✅
[giza][2024-06-09 13:25:16.477] Version Created with id -> 1! ✅
[giza][2024-06-09 13:25:16.479] Sending model for transpilation ✅ 
[giza][2024-06-09 13:26:03.560] Transpilation is fully compatible. Version compiled and Sierra is saved at Giza ✅
⠴ Transpiling Model...
[giza][2024-06-09 13:26:04.916] Downloading model ✅
[giza][2024-06-09 13:26:04.923] model saved at: verifiable_lr


In [5]:
!giza endpoints deploy --model-id 764 --version-id 1

▰▱▱▱▱▱▱ Creating endpoint!t!
[giza][2024-06-09 13:26:21.312] Endpoint is successful ✅
[giza][2024-06-09 13:26:21.314] Endpoint created with id -> 332 ✅
[giza][2024-06-09 13:26:21.315] Endpoint created with endpoint URL: https://endpoint-ashu-764-1-7c6e9a08-7i3yxzspbq-ew.a.run.app 🎉


In [6]:
from giza.agents.model import GizaModel

MODEL_ID = 764  # Update with your model ID
VERSION_ID = 1  # Update with your version ID

def prediction(input, model_id, version_id):
    model = GizaModel(id=model_id, version=version_id)

    (result, proof_id) = model.predict(
        input_feed={'input': input}, verifiable=True
    )

    return result, proof_id

def execution():
    # The input data type should match the model's expected input
    input = np.array([[34.5]]).astype(np.float32)

    (result, proof_id) = prediction(input, MODEL_ID, VERSION_ID)

    print(
        f"Predicted value for input {input.flatten()[0]} is {result[0].flatten()[0]}")

    return result, proof_id


execution()

/Users/ashu/.pyenv/versions/3.11.0/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Payload:  Tensor<FP16x16>
🚀 Starting deserialization process...
✅ Deserialization completed! 🎉
Predicted value for input 34.5 is 0.9709320068359375


(array([[0.97093201]]), 'f08ef5ccd5ca4230a362b843fbc9a7e0')

In [7]:
!giza endpoints get-proof --endpoint-id 332 --proof-id "f08ef5ccd5ca4230a362b843fbc9a7e0"

[giza][2024-06-09 13:27:27.776] Getting proof from endpoint 332 ✅ 
{
  "id": 1132,
  "job_id": 1306,
  "metrics": {
    "proving_time": 18.351759
  },
  "created_date": "2024-06-09T07:57:19.773216"
}


In [8]:
! giza endpoints download-proof --endpoint-id 332 --proof-id "f08ef5ccd5ca4230a362b843fbc9a7e0" --output-path zklr.proof

[giza][2024-06-09 13:28:16.322] Getting proof from endpoint 332 ✅ 
[giza][2024-06-09 13:28:21.777] Proof downloaded to zklr.proof ✅ 


In [9]:
!giza verify --proof-id 332

[giza][2024-06-09 13:28:36.240] Verifying proof...
[giza][2024-06-09 13:28:38.359] Verification result: True
[giza][2024-06-09 13:28:38.359] Verification time: 0.448411401
